In [3]:
import jax
import numpy as np
from neurallogic import hard_not, hard_and, harden, neural_logic_net, harden_layer, primitives
from tests import test_mnist

In [5]:
train_data, test_data = test_mnist.get_datasets()

2022-11-17 16:02:09.407866: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 47040000 exceeds 10% of free system memory.


In [6]:
len(train_data)

2

: 